In [ ]:
! pip install --quiet "xlrd" "scikit-learn" "ipython[notebook]==7.34.0, <8.17.0" "openpyxl" "fastparquet" "lightgbm" "pyarrow" "setuptools>=68.0.0, <68.3.0"  "xgboost" "catboost" "tensorboard" "lightning>=2.0.0" "urllib3" "torch==2.3.0" "matplotlib" "optuna" "pytorch-lightning>=1.4, <2.1.0" "seaborn" "torchvision" "torchmetrics>=0.7, <1.3" "matplotlib>=3.0.0, <3.9.0" "numpy==1.26.4"

In [ ]:
import os

import lightning as L
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
from torchvision import transforms
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

TRAIN_IMAGES_DATA_PATH = "./train_images"
TEST_IMAGES_DATA_PATH = "./test_images"
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/")

plt.set_cmap("cividis")
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("png", "svg", "pdf")  # For export
sns.reset_orig()

L.seed_everything(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
from torch.utils.data import DataLoader
from data_set import PlantDataset

from sklearn.preprocessing import StandardScaler

img_transform_train = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img_transform_test = test_transform = transforms.Compose(
    [
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# do not normalize CSV features until split to avoid data leakage
partial_training_data = PlantDataset('train.csv', 'train_images', num_labels=6, image_transform=img_transform_train)
partial_test_data = PlantDataset('test.csv', 'test_images', num_labels=0, image_transform=img_transform_test)


In [ ]:
from data_set import AugmentedDataset

# Augment data sets

## Change this when regenerating embeddings
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg').to(device)
# model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg').to(device)
full_training_data = AugmentedDataset(partial_training_data, model, "train_embeddings.parquet", device=device)
test_data = AugmentedDataset(partial_test_data, model, "test_embeddings.parquet", device=device)

In [ ]:
from data_set import PandasDataset

class MLPModel(L.LightningModule):
    def __init__(self, input_dim=1699, output_dim=6, lr=5e-4, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.body = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.GELU(),
            nn.Linear(1024, 256),
            nn.GELU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, row):
        x = self.body(row)
        return x

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=0.0005)
        return [optimizer], []

    def _calculate_loss(self, batch, mode="train"):
        rows, labels = batch

        preds = self.forward(rows)
        loss = F.mse_loss(torch.squeeze(preds), torch.squeeze(labels))
        self.log(f"{mode}_loss", loss)
        self.log(f"{mode}_r2", r2_score(labels.cpu().numpy(), preds.detach().cpu().numpy()))
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="train")

    def validation_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        pass
    
    def predict(self, X : pd.DataFrame):
        with torch.no_grad():
            return self.forward(torch.tensor(X.values, dtype=torch.float32).to(device)).cpu().numpy()
    
    def score(self, X : pd.DataFrame, Y: pd.DataFrame):
        return r2_score(Y, self.predict(X))

class MyDataModule(L.LightningDataModule):
    def __init__(self, X_train, Y_train, X_val, Y_val, batch_size=512):
        super().__init__()
        self.mlp_train_set = PandasDataset(X_train, Y_train)
        self.mlp_val_set = PandasDataset(X_val, Y_val)
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.mlp_train_set, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.mlp_val_set, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        pass


import os
def train_mlp_model(X_train, Y_train, X_val, Y_val, batch_size=256, dry_run=False, run_num=0, **kwargs):
    mlp_data_module = MyDataModule(X_train, Y_train, X_val, Y_val, batch_size=batch_size)
    trainer = L.Trainer(
        default_root_dir=os.path.join(CHECKPOINT_PATH, f"{run_num}/mlp/"),
        accelerator="auto",
        devices=1,
        max_epochs=5 if not dry_run else 1,
        callbacks=[
            ModelCheckpoint(save_weights_only=True, mode="max", monitor="epoch"),
            EarlyStopping(monitor='val_r2', patience=1, mode="max"),
            LearningRateMonitor("epoch"),
        ],
        enable_progress_bar=False
    )
    trainer.logger._log_graph = True
    trainer.logger._default_hp_metric = None

    model = MLPModel(
        input_dim=X_train.shape[1],
        output_dim=Y_val.shape[1] if not dry_run else 1,
        **kwargs
    )
    trainer.fit(model, datamodule=mlp_data_module)

    # Load the best checkpoint after training
    model = MLPModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

    return model

In [ ]:
# Opens tensorboard in notebook. Adjust the path to your CHECKPOINT_PATH!
%reload_ext tensorboard
%tensorboard --logdir ./saved_models


In [ ]:
from sklearn.model_selection import KFold, ShuffleSplit
import xgboost
import catboost
import lightgbm

from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.multioutput import MultiOutputRegressor

from plant_trait_regressor import PlantTraitRegressor

# do a quick run through
dry_run = False

kf = ShuffleSplit(
    n_splits=5 if not dry_run else 1, 
    random_state=42, 
    test_size=0.1
)
preds_test = np.zeros((len(test_data), 6 if not dry_run else 1))

if dry_run:
    print ("Dry running code...")


for i, (train_index, test_index) in enumerate(kf.split(full_training_data.csv_aug, full_training_data.labels)):
    # train sets
    X_train = full_training_data.csv_aug.iloc[train_index].copy()
    Y_train = full_training_data.labels.iloc[train_index].copy()
    
    # validation sets
    X_val = full_training_data.csv_aug.iloc[test_index].copy()
    Y_val = full_training_data.labels.iloc[test_index].copy()
    
    if dry_run:
        Y_train = pd.DataFrame(Y_train.iloc[:, 0])
        Y_val = pd.DataFrame(Y_val.iloc[:, 0])
    
    # Test set
    X_test = test_data.csv_aug.copy()
    
    # for boosting algorithms, it can be beneficial to engineer some features
    poly = PolynomialFeatures(2)

    # Randomly select 1000 extra polynomial features
    num_extra_features = 1000
    poly.fit(X_train.iloc[:, :163])
    random_extra_features = np.random.choice(range(163, poly.n_output_features_), num_extra_features, replace=False)
    
    # Augment each of the corresponding feature sets
    X_train_extra_features = pd.DataFrame(np.concatenate((X_train.values, poly.transform(X_train.iloc[:, :163])[:, random_extra_features]), axis=1))
    X_val_extra_features = pd.DataFrame(np.concatenate((X_val.values,  poly.transform(X_val.iloc[:, :163])[:, random_extra_features]), axis=1))
    X_test_extra_features = pd.DataFrame(np.concatenate((X_test.values, poly.transform(X_test.iloc[:, :163])[:, random_extra_features]), axis=1))
    
    columns_no_embed = X_train_extra_features.columns

    # for catboost, add embeddings
    X_train_extra_features['emb'] = list(X_train.iloc[:, 163:].values)
    X_val_extra_features['emb'] = list(X_val.iloc[:, 163:].values)
    X_test_extra_features['emb'] = list(X_test.iloc[:, 163:].values)
    
    # For the other models, need to normalize
    X_pipeline = Pipeline([('scaler', RobustScaler())])
    Y_pipeline = Pipeline([('scaler', StandardScaler())])

    X_train[X_train.columns] = X_pipeline.fit_transform(X_train)
    Y_train[Y_train.columns] = Y_pipeline.fit_transform(Y_train)

    X_val[X_val.columns] = X_pipeline.transform(X_val)
    Y_val[Y_val.columns] = Y_pipeline.transform(Y_val)

    X_test[X_test.columns] = X_pipeline.transform(X_test)
    
    best_xgb = {
        "objective": "reg:squarederror",
        "n_estimators": 1000 if not dry_run else 1,
        "learning_rate": 0.029604246449770312, 
        "max_depth": 8, 
        "subsample": 0.8080014405993786, 
        "colsample_bytree": 0.6684075982840267, 
        "min_child_weight": 20
    }
    xgb = MultiOutputRegressor(
        xgboost.XGBRegressor(
            **best_xgb
        ),
        n_jobs=3 if not dry_run else 1
    )
    print("Xgb: ", 
          xgb.fit(
              X_train_extra_features[columns_no_embed], 
              Y_train, verbose=False
            ).score(X_val_extra_features[columns_no_embed], Y_val))
    
    best_cat = {'learning_rate': 0.05, 'depth': 9}
    cat = PlantTraitRegressor(
        catboost.CatBoostRegressor(
            iterations=2000 if not dry_run else 1,
            embedding_features=["emb"],
            eval_metric="R2",
            early_stopping_rounds=1000,
            use_best_model=True,
            verbose=False,
            **best_cat
        ),
        n_jobs=2 if not dry_run else 1
    )
    print("Cat: ", cat.fit(X_train_extra_features, Y_train, eval_set=(X_val_extra_features, Y_val)).score(X_val_extra_features, Y_val))
    
    best_lgb = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 1500 if not dry_run else 1,
        "bagging_freq": 1,
        "learning_rate": 0.010144890360462996,
        "num_leaves": 724,
        "subsample": 0.9896282659716074,
        "colsample_bytree": 0.2884524600576782,
        "min_data_in_leaf": 61,
        "verbosity": -1,
    }
    lgb = PlantTraitRegressor(
        lightgbm.LGBMRegressor(
            linear_tree = True,
            **best_lgb
        ),
        n_jobs=2 if not dry_run else 1
    )
    print("Lgb: ", lgb.fit(
        X_train_extra_features[columns_no_embed], Y_train, 
        eval_set=(X_val_extra_features[columns_no_embed], Y_val), 
        callbacks=[lightgbm.early_stopping(stopping_rounds=100)]
    ).score(X_val_extra_features[columns_no_embed], Y_val))
    
    ridge = Ridge()
    print("Ridge: ", ridge.fit(X_train, Y_train).score(X_val, Y_val))

    reg = KNeighborsRegressor(
        n_neighbors=7 if not dry_run else 1, 
        metric="manhattan", 
        weights='distance'
    )
    print("Reg: ", reg.fit(X_train, Y_train).score(X_val, Y_val))
    
    mlp = train_mlp_model(
        X_train=X_train,
        Y_train=Y_train,
        X_val=X_val,
        Y_val=Y_val,
        batch_size=256,
        lr=5e-4,
        dry_run=dry_run,
        run_num=i,
    )
    print ("MLP: ", mlp.score(X_val, Y_val))
    
    l_train_X = np.column_stack((
        Y_pipeline.transform(xgb.predict(X_val_extra_features[columns_no_embed])), 
        Y_pipeline.transform(cat.predict(X_val_extra_features)), 
        Y_pipeline.transform(lgb.predict(X_val_extra_features[columns_no_embed])), 
        ridge.predict(X_val), 
        reg.predict(X_val), 
        mlp.predict(X_val))
    )
    
    meta = Lasso(alpha=0.00006)
    print (f"Done: {i} with score", meta.fit(l_train_X, Y_val).score(l_train_X, Y_val))

    l_test_X = np.column_stack((
        Y_pipeline.transform(xgb.predict(X_test_extra_features[columns_no_embed])), 
        Y_pipeline.transform(cat.predict(X_test_extra_features)), 
        Y_pipeline.transform(lgb.predict(X_test_extra_features[columns_no_embed])), 
        ridge.predict(X_test), 
        reg.predict(X_test), 
        mlp.predict(X_test))
    )

    if dry_run:
        preds = Y_pipeline.inverse_transform(meta.predict(l_test_X).reshape(-1, 1))
    else:
        preds = Y_pipeline.inverse_transform(meta.predict(l_test_X))
    preds_test += preds / kf.get_n_splits()
    
    print ("================\n\n")

In [ ]:
r2_score(Y_pipeline.inverse_transform(xgb.predict(X_test_extra_features[columns_no_embed])), preds_test)

In [ ]:
import csv
with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
    for i in range(len(preds_test)):
      writer.writerow([test_data.plant.ids[i], preds_test[i][0], preds_test[i][1], preds_test[i][2], preds_test[i][3], preds_test[i][4], preds_test[i][5]])